## Case Study: Prediction of Traffic Volume

<img src="traffic_image.gif" width=500 height = 280/>

### Import Libraries

In [1]:
import pandas as pd                  # Pandas
import numpy as np                   # Numpy
from matplotlib import pyplot as plt # Matplotlib

# Package to implement ML Algorithms
import sklearn
from sklearn.ensemble import RandomForestRegressor # Random Forest

# Import MAPIE to calculate prediction intervals
from mapie.regression import MapieRegressor

# To calculate coverage score
from mapie.metrics import regression_coverage_score

# Package for data partitioning
from sklearn.model_selection import train_test_split

# Package to record time
import time

# Module to save and load Python objects to and from files
import pickle 

# Ignore Deprecation Warnings
import warnings
warnings.filterwarnings('ignore')

# Display inline plots as vector-based (svg)
%config InlineBackend.figure_formats = ['svg']

%matplotlib inline

Install XGBoost

In [2]:
import xgboost as xgb

**Install MAPIE via `pip`**:

`!pip install mapie`

In [3]:
%pip install mapie==0.9.1

Note: you may need to restart the kernel to use updated packages.


### Load Dataset

In [4]:
# Import Data
df = pd.read_csv('Traffic_Volume.csv')
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,date_time,traffic_volume
0,NaN,288.28,0.0,0.0,40,Clouds,10/2/12 9:00,5545
1,NaN,289.36,0.0,0.0,75,Clouds,10/2/12 10:00,4516
2,NaN,289.58,0.0,0.0,90,Clouds,10/2/12 11:00,4767
3,NaN,290.13,0.0,0.0,90,Clouds,10/2/12 12:00,5026
4,NaN,291.14,0.0,0.0,75,Clouds,10/2/12 13:00,4918


In [5]:
# Convert to datetime
df['date_time'] = pd.to_datetime(df['date_time'], format='%m/%d/%y %H:%M')

# Extract features
# Since I do not want the date_time column, I extracted all the features and made a new column for each feature
# I do not need to include year or day, so I dropped it later on
# I used ChatGPT here to help me with the syntax for extracting month name and weekday name
df['holiday'] = df['holiday'].fillna("None")
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month_name() # I used ChatGPT here to extract the month name
df['day'] = df['date_time'].dt.day
df['weekday'] = df['date_time'].dt.day_name() # I used ChatGPT here to extract the weekday name
df['hour'] = df['date_time'].dt.hour

# Drop original date column
df = df.drop(columns=['date_time'])

# Select input and output features
X = df.drop(columns = ['traffic_volume', 'year', 'day'])
y = df['traffic_volume']

# One-hot encode categorical variables
categorical_cols = X.select_dtypes(include=['object']).columns
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=False)

X_encoded.head()

,temp,rain_1h,snow_1h,clouds_all,hour,holiday_Christmas Day,holiday_Columbus Day,holiday_Independence Day,holiday_Labor Day,holiday_Martin Luther King Jr Day,...,month_November,month_October,month_September,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,288.28,0.0,0.0,40,9,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
1,289.36,0.0,0.0,75,10,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
2,289.58,0.0,0.0,90,11,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3,290.13,0.0,0.0,90,12,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,291.14,0.0,0.0,75,13,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False


In [6]:
list(X_encoded.columns)

['temp',
 'rain_1h',
 'snow_1h',
 'clouds_all',
 'hour',
 'holiday_Christmas Day',
 'holiday_Columbus Day',
 'holiday_Independence Day',
 'holiday_Labor Day',
 'holiday_Martin Luther King Jr Day',
 'holiday_Memorial Day',
 'holiday_New Years Day',
 'holiday_None',
 'holiday_State Fair',
 'holiday_Thanksgiving Day',
 'holiday_Veterans Day',
 'holiday_Washingtons Birthday',
 'weather_main_Clear',
 'weather_main_Clouds',
 'weather_main_Drizzle',
 'weather_main_Fog',
 'weather_main_Haze',
 'weather_main_Mist',
 'weather_main_Rain',
 'weather_main_Smoke',
 'weather_main_Snow',
 'weather_main_Squall',
 'weather_main_Thunderstorm',
 'month_April',
 'month_August',
 'month_December',
 'month_February',
 'month_January',
 'month_July',
 'month_June',
 'month_March',
 'month_May',
 'month_November',
 'month_October',
 'month_September',
 'weekday_Friday',
 'weekday_Monday',
 'weekday_Saturday',
 'weekday_Sunday',
 'weekday_Thursday',
 'weekday_Tuesday',
 'weekday_Wednesday']

In [7]:
unique_holidays = df['holiday'].unique()
print(unique_holidays)

['None' 'Columbus Day' 'Veterans Day' 'Thanksgiving Day' 'Christmas Day'
 'New Years Day' 'Washingtons Birthday' 'Memorial Day' 'Independence Day'
 'State Fair' 'Labor Day' 'Martin Luther King Jr Day']


In [8]:
unique_weather = df['weather_main'].unique()
print(unique_weather)

['Clouds' 'Clear' 'Rain' 'Drizzle' 'Mist' 'Haze' 'Fog' 'Thunderstorm'
 'Snow' 'Squall' 'Smoke']


In [9]:
# Data partitioning into train and test sets
train_X, test_X, train_y, test_y = train_test_split(X_encoded, y, test_size = 0.2, random_state = 42)

### Implement XGBoost

In [10]:
# Define your model
reg = xgb.XGBRegressor(random_state = 42)

In [11]:
# Fit the model
start = time.time()            # Start Time
reg.fit(train_X, train_y)  
stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

Training time: 0.2598440647125244s


#### Evaluate Prediction Performance on Test Set

In [12]:
# Predict Test Set
y_pred = reg.predict(test_X)

# Evaluate the model on test set
r2 = sklearn.metrics.r2_score(test_y, y_pred)
print('R-squared on Test Set: %0.2f' %r2)

RMSE_test = sklearn.metrics.root_mean_squared_error(test_y, y_pred)
print('RMSE on Test Set: %0.2f' %RMSE_test)

# Predict Train Set
y_train_pred = reg.predict(train_X)

# Evaluate the model on train set
r2_train = sklearn.metrics.r2_score(train_y, y_train_pred)
print('R-squared on Train Set: %0.2f' %r2_train)

RMSE_train = sklearn.metrics.root_mean_squared_error(train_y, y_train_pred)
print('RMSE on Train Set: %0.2f' %RMSE_train)

R-squared on Test Set: 0.95
RMSE on Test Set: 430.04
R-squared on Train Set: 0.97
RMSE on Train Set: 370.87


#### Histogram of Residuals/Errors 

In [13]:
# Calculate the residuals by subtracting the predicted values from the actual test values
all_residuals = test_y - y_pred

# Set up the figure with custom size and resolution (DPI)
plt.figure(figsize=(6, 4), dpi = 150)

# Plot the histogram of residuals
plt.hist(all_residuals, bins = 25, color = 'green', edgecolor = 'black')

# Label X and Y axes
plt.xlabel('Residuals', fontsize = 14)
plt.ylabel('# of Test Datapoints', fontsize = 14)

# Set the title of the plot
plt.title('Distribution of Residuals', fontsize = 16)

# Adjust the font size of x and y ticks
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10);

plt.savefig("all_residuals.svg", format='svg', bbox_inches='tight')
plt.close()

#### Scatter Plot of Predicted Vs. Actual Values

In [14]:
# Setting the figure size and resolution
plt.figure(figsize = (6, 4), dpi = 150)

# Scatter plot of actual vs predicted values
plt.scatter(test_y, y_pred, color = 'lightblue', alpha = 0.6, edgecolor = 'black', s = 40)

# 45-degree reference line (perfect predictions)
plt.plot([min(test_y), max(test_y)], [min(test_y), max(test_y)], color = 'red', linestyle = '--', lw = 2)

# Axis labels and title
plt.xlabel('Actual Values', fontsize = 10)
plt.ylabel('Predicted Values', fontsize = 10)
plt.title('Predicted vs Actual Values', fontsize = 12)

# Adjust ticks
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)

plt.savefig("predicted_vs_actual.svg", format='svg', bbox_inches='tight')
plt.close()

#### Feature Importance Plot

In [15]:
# Storing importance values from the trained model
importance = reg.feature_importances_

# Storing feature importance as a dataframe
feature_imp = pd.DataFrame(list(zip(train_X.columns, importance)),
               columns = ['Feature', 'Importance'])

feature_imp = feature_imp.sort_values('Importance', ascending = False).reset_index(drop = True)

# Bar plot
plt.figure(figsize=(8, 4))
plt.barh(feature_imp['Feature'], feature_imp['Importance'], color = ['green', 'lightblue'])

plt.xlabel("Importance", fontsize = 12)
plt.ylabel("Input Feature", fontsize = 12)
plt.yticks(fontsize = 5) # fontsize of yticks
plt.xticks(fontsize = 10) # fontsize of xticks

plt.tight_layout()
plt.savefig("feature_imp.svg", format='svg', bbox_inches='tight')
plt.close()

#### **Prediction Intervals using MAPIE Regressor**

In [16]:
# Define MAPIE regressor
mapie = MapieRegressor(estimator = reg, # Prediction model to use
                       n_jobs = -1,
                       random_state = 42)

# Fit mapie regressor on training data
start = time.time()  
mapie.fit(train_X, train_y)
stop = time.time()             
print(f"Training time: {stop - start}s")

alpha = 0.1 # For 90% confidence level

# Use mapie.predict() to get predicted values and intervals
y_test_pred, y_test_pis = mapie.predict(test_X, alpha = alpha)

Training time: 2.549643039703369s


y_test_pred will give point prediction

In [17]:
# Predicted values
y_test_pred 

array([5845.0996 , 2216.2437 ,  682.71136, ..., 2002.1223 , 2888.0327 ,
        625.2689 ], shape=(9641,), dtype=float32)

In [18]:
# Prediction Intervals
y_test_pis

array([[[5139.76794434],
        [6288.49487305]],

       [[1686.74316406],
        [2835.88964844]],

       [[ 111.06610107],
        [1257.58856201]],

       ...,

       [[1481.60229492],
        [2646.85473633]],

       [[2272.20239258],
        [3424.11376953]],

       [[  26.12677002],
        [1172.86627197]]], shape=(9641, 2, 1))

In [19]:
# Storing results in a dataframe
predictions = test_y.to_frame()
predictions.columns = ['Actual Value']
predictions["Predicted Value"] = y_test_pred.round(2)
predictions["Lower Value"] = y_test_pis[:, 0].round(2)
predictions["Upper Value"] = y_test_pis[:, 1].round(2)

# Take a quick look
predictions.tail(5)

,Actual Value,Predicted Value,Lower Value,Upper Value
6401,2491,2714.030029,2111.51,3267.33
34004,5251,5203.580078,4659.40,5805.01
46086,2224,2002.119995,1481.60,2646.85
42579,2928,2888.030029,2272.20,3424.11
23709,910,625.270020,26.13,1172.87


### **Coverage Calculation**
- **Coverage** refers to the proportion of true/actual values that fall within the prediction intervals generated by a model.

- It is a measure of how well the prediction intervals capture the actual values.

  $\text{Coverage} = \frac{\text{Number of actual values within prediction intervals}}{\text{Total number of actual values}}$

In [20]:
coverage = regression_coverage_score(test_y,           # Actual values
                                     y_test_pis[:, 0], # Lower bound of prediction intervals
                                     y_test_pis[:, 1]) # Upper bound of prediction intervals

coverage_percentage = coverage * 100
print(f"Coverage: {coverage_percentage:.2f}%")

Coverage: 91.58%


In [21]:
# Sort the predictions by 'Actual Value' for better visualization and reset the index
sorted_predictions = predictions.sort_values(by=['Actual Value']).reset_index(drop=True)

# Create a figure and axis object with specified size and resolution
fig, ax = plt.subplots(figsize=(8, 4))

# Plot the actual values with green dots
plt.plot(sorted_predictions["Actual Value"], 'go', markersize=3, label="Actual Value")

# Fill the area between the lower and upper bounds of the prediction intervals with semi-transparent blue color
plt.fill_between(np.arange(len(sorted_predictions)),
                 sorted_predictions["Lower Value"],
                 sorted_predictions["Upper Value"],
                 alpha=0.2, color="blue", label="Prediction Interval")

# Set font size for x and y ticks
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Set the limit for the x-axis to cover the range of samples
plt.xlim([0, len(sorted_predictions)])

# Label the x-axis and y-axis with appropriate font size
plt.xlabel("Samples", fontsize=10)
plt.ylabel("Target", fontsize=10)

# Add a title to the plot, including the coverage percentage, with bold formatting
plt.title(f"Prediction Intervals and Coverage: {coverage_percentage:.2f}%", fontsize=12, fontweight="bold")

# Add a legend to the plot, placed in the upper left, with specified font size
plt.legend(loc="upper left", fontsize=10)

plt.savefig("coverage_plot.svg", format='svg', bbox_inches='tight')
plt.close()

### Save Model

In [22]:
# Creating the file where we want to write the model
reg_pickle = open('traffic_volume.pickle', 'wb') 

# Write RF model to the file
pickle.dump(mapie, reg_pickle) 

# Close the file
reg_pickle.close()